# Indice CHF (Extensión)

## Introducción

En este trabajo, nos centramos en la creación de un índice esencial para comprender la climatología de huracanes en las regiones de Costa Rica y Nicaragua: el CHF (Coastal Hurricane Frequency). Esta métrica se basa en la técnica CHF, como se plantea en el artículo de Karthik Balaguru, "Increased U.S. coastal hurricane risk under climate change". Sin embargo, es importante destacar que hemos realizado modificaciones significativas en este índice. La versión que empleamos ha sido adaptada y ampliada por Luis Barboza, Shu Wei Chou y Mario Gomez para abordar tanto el impacto directo como el indirecto de los ciclones tropicales en estas regiones.

Este índice se encuentra en una fase preliminar y es objeto de constante refinamiento en el marco de esta investigación y tiene la siguiente forma:

$$
CHF_{m} = \textrm{Efecto Directo} + \textrm{Efecto Indirecto}
$$

$$
CHF_{m} = CHF + N_{i} \frac{l}{TimeStep*v} \frac{P}{\overline P_{r}} \frac{Viento}{\overline{Viento_{r}}}
$$

$$
CHF_{m} = n\frac{l}{TimeStep*v} + N_{i} \frac{l}{TimeStep*v} \frac{P}{\overline P_{r}} \frac{Viento}{\overline{Viento_{r}}}
$$
En donde, 
$$
n: \textrm{es el número de trayectorias de huracanes que pasan a través de un grado cuadrado}
$$ 
$$
N_i :\textrm{Cantidad de Huracanés que no pasan a traves de esa cuadrícula}
$$
$$
l: \textrm{la longitud de la cuadrícula}
$$
$$
v: \textrm{la velocidad de traslación}
$$
$$
TimeStep: \textrm{es el paso de tiempo}
$$
$$
P: \textrm{Precipitación en esa cuadrícula}
$$
$$
\overline P_{r}: \textrm{Precipitación promedio en toda la región}
$$
$$
Viento: \textrm{ Viento en la cuadrícula }
$$
$$
\overline {Viento_{r}} : \textrm {Viento promedio en toda la región}
$$


## Metodología 

Se utilizara Python para el análisis de este cuaderno, en donde se utilizaron las siguientes librerias

### Librerias

-   **NumPy:** Libreria para realizar cálculos numéricos en Python. Es ampliamente utilizada para operaciones matemáticas y de manipulación de datos.

-   **Pandas:** Pandas es una biblioteca que proporciona estructuras de datos y herramientas para el análisis de datos. Es especialmente útil para la manipulación y exploración de datos tabulares.

-   **GeoPandas:** GeoPandas es una extensión de Pandas que agrega capacidades geoespaciales. Permite trabajar con datos geoespaciales, como GeoDataFrames, y realizar análisis espaciales.

-   **OS:** El módulo `os` proporciona funciones para interactuar con el sistema operativo, como la manipulación de rutas de archivos y directorios.

-   **Shapely:** Shapely es una biblioteca que se utiliza para el procesamiento y análisis de geometría plana. Permite trabajar con objetos geométricos como puntos, líneas y polígonos.

-   **pyogrio:** pyogrio es una biblioteca que se utiliza para leer y escribir datos geoespaciales en formatos como GeoJSON y Shapefile. Facilita la manipulación de datos geoespaciales.

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os 
import shapely
from pyogrio import read_dataframe
from shapely.geometry import Polygon, Point|

## Creación del Indicador CHF

En esta sección, llevaremos a cabo la implementación del CHF modificado, abordando tanto el impacto directo como el impacto indirecto, siguiendo la solicitud de los investigadores. Además, exploraremos el índice CHF, que engloba tanto el impacto directo como el indirecto de los ciclones tropicales en la región.

Es importante mencionar que, debido a ciertas limitaciones en la obtención de datos de la página de ERA5 (https://cds.climate.copernicus.eu/cdsapp#!/home), así como en la base del HURDAT, habrá años para los cuales no se dispone de información. Sin embargo, estas limitaciones, en su mayoría asociadas a huracanes más antiguos, no deberían tener un impacto significativo en nuestro análisis.

### Importación de Bases de Datos

En esta etapa, procederemos a importar diversas bases de datos esenciales para nuestro análisis. En primer lugar, incorporaremos la base de datos de Natural Earth (https://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/), que abarca alrededor de 258 países, proporcionando información geográfica clave.

Asimismo, importaremos la base de datos del HURDAT, la cual está condicionada a un intervalo de tiempo de 24 horas, y es fundamental para nuestro estudio de huracanes.

Finalmente, incorporaremos la base de datos de ERA5, que nos permitirá enriquecer nuestro análisis con información sobre la velocidad de los vientos y la precipitación, añadiendo una dimensión adicional a nuestra investigación.

In [4]:
paises = read_dataframe("Shapefiles/ne_110m_admin_0_countries.shp")

In [5]:
hurdat = pd.read_csv("CHF_Balagurù/Datos/hurdat2daily.csv")

In [6]:
precip = pd.read_csv("precip.csv")

In [7]:
wind = pd.read_csv("wind.csv")

### Filtración y Manipulación de las Bases

En esta sección, se aborda el proceso clave de filtrar y ajustar las bases de datos para adaptarlas a los requisitos específicos del estudio. En primer lugar, se limita la base del HURDAT a datos posteriores a 2010, enfocándose exclusivamente en los países de Centroamérica y México para afinar el enfoque geográfico.

Posteriormente, se procede con la simplificación de la base de vientos. Se reduce la complejidad de las coordenadas, tomando solo cada unidad de latitud y longitud en lugar de cada decimal. Esta medida facilita la manipulación de la base, permitiendo una extracción más eficiente de patrones significativos para el análisis climatológico.

In [8]:
paises_interes = ["Costa Rica","Nicaragua","Belize","Panama","El Salvador","Mexico","Honduras","Guatemala"]

paises = paises[["SOVEREIGNT","geometry"]]
paises = paises[paises['SOVEREIGNT'].isin(paises_interes)]

Se guarda la base de datos de paises en formato geojson para su futura implementación en futuros gráficos, para R y Tableau.

In [143]:
with open('paises.geojson' , 'w') as file:
    file.write(paises.to_json())

Posteriormente de guardar la base de datos de los paises de Centro América en formato geojson, se procede a filtrar la base solo a las fechas en las cuales se presentan los huracanes; así se puede realizar los cálculos adecuadamente. Además se filtra la base de datos del hurdat para 2010 en adelante para no tener una base tan extensa y para no calcular erroneamente las frecuencias de los huracanes por cuadrícula.

In [9]:
wind = wind[wind['Time'].isin(hurdat['dat'])]

In [10]:
precip = precip[precip['Time'].isin(hurdat['dat'])]

In [81]:
hurdat[["year","month","day"]] = hurdat["dat"].str.split('-',expand=True) #Separamos la fecha
hurdat = hurdat.astype({'year':'int'})

hurdat = hurdat[hurdat['year']>=2010]

Seguidamente de filtrar la base en las fechas correctas, se procede a delimitar la base de datos del Hurdat, Precipitación y Viento, para poder tener los limites de las coordenadas correctas.

In [11]:
minx, miny, maxx, maxy = paises.total_bounds #Limites de los paises

In [82]:
#Delimitamos la zona de la base de HurDat para las zonas a analizar

huracanes = hurdat[((hurdat["Lon"]<= maxx)&
                            (hurdat["Lon"]>= minx))&
                            ((hurdat["Lat"]<= maxy)&
                            (hurdat["Lat"]>= miny))] #Delimitación de la zona

#Se convierte la base de huracanes en formato GeoDataFrame
huracanes = gpd.GeoDataFrame(huracanes,
                                geometry=gpd.points_from_xy(huracanes.Lon, huracanes.Lat), 
                                crs="EPSG:4326") #Base del Hurdat con formato geoespacial

#Filtramos la base de huracanes para reducir espacio a las variables necesarias para el cálculo del CHF
huracanes = huracanes[['Name','MaxW','geometry']]

In [13]:
#Delimitamos la zona de la base de precipitación para las zonas a analizar

precipitacion = precip[((precip["Longitude"] <= maxx)&
                       (precip["Longitude"] >= minx))&
                      ((precip["Latitude"] <= maxy)&
                      (precip["Latitude"] >= miny))] #Delimitación de la zona

In [14]:
#Se convierte la base de precipitación en formato GeoDataFrame

precipitacion = gpd.GeoDataFrame(precipitacion,
                                geometry=gpd.points_from_xy(precipitacion.Longitude, precipitacion.Latitude), 
                                crs="EPSG:4326") #Base del Hurdat con formato geoespacial

#Filtramos la base de precipitación para reducir espacio a las variables necesarias para el cálculo del CHF
precipitacion = precipitacion[['Precipitation','geometry']]

In [17]:
#Delimitamos la zona de la base de precipitación para las zonas a analizar

wind = wind[((wind["Longitude"] <= maxx)&
                       (wind["Longitude"] >= minx))&
                      ((wind["Latitude"] <= maxy)&
                      (wind["Latitude"] >= miny))] #Delimitación de la zona

**Nota sobre la Depuración de la Base de Datos**

Durante el proceso de análisis de la base de datos, se identificó un desafío en la comparación de valores en la columna 'Latitude'. La gran cantidad de datos en la base puede afectar la precisión de las operaciones numéricas, especialmente al tratar con valores de punto flotante.

El enfoque final utilizado implica la conversión de la columna 'Latitude' a cadena y la verificación de la presencia de ".0" en la cadena. Esto permite mantener las filas donde 'Latitude' es esencialmente un número entero, independientemente de su representación exacta como decimal.

Es importante destacar que este proceso de depuración se llevó a cabo para garantizar resultados precisos y confiables en el análisis de los datos.


In [20]:
wind = wind[wind['Latitude'].astype(str).str.contains("\.0")]

In [21]:
wind = wind[wind['Longitude'].astype(str).str.contains("\.0")]

In [24]:
#Se convierte la base de viento en formato GeoDataFrame

wind = gpd.GeoDataFrame(wind,
                        geometry=gpd.points_from_xy(wind.Longitude, wind.Latitude), 
                                crs="EPSG:4326") #Base del Hurdat con formato geoespacial

In [26]:
#Filtramos la base de viento para reducir espacio a las variables necesarias para el cálculo del CHF
wind = wind[['Precipitation','geometry']] 


wind.rename({'Precipitation':'Wind'},axis=1,inplace=True) # Se renombra la columna ya que en el otro ipynb no se puso el nombre correcto

## Creación del Indicador CHF: Version Antigua y Versiones Nuevas

En esta sección, se detalla el proceso de creación del indicador CHF, abarcando tanto su versión original propuesta por Balaguru como la nueva implementación ajustada para adaptarse a las particularidades de la base de datos del HURDAT y a los requisitos específicos de la investigación.

Primero, se describe la metodología empleada para calcular el CHF original propuesto por Balaguru, destacando las variables clave y la fórmula utilizada. A continuación, se presenta la nueva versión del CHF, que incorpora ajustes necesarios para garantizar su aplicabilidad a la base de datos del HURDAT y considera las condiciones geográficas de la región de estudio.

Es importante señalar que, durante el proceso de implementación del CHF en su versión ajustada, se identificó una inconsistencia en la fórmula que calcula la velocidad en el efecto indirecto. Con el objetivo de abordar esta irregularidad, se propuso un nuevo CHF extra. Cabe mencionar que este nuevo enfoque debe ser revisado por los investigadores antes de su aplicación, ya que su validez y coherencia son fundamentales para los resultados obtenidos.

### Primera Parte
Primero se crean las cuadrículas utilizadas en el CHF propuesto por Balagurù, estas son de un grado cuadrado

In [28]:
grado_cuad = 1 #Grado cuadrado = metro cuadrado

x_coords = np.arange(minx, maxx, grado_cuad)
y_coords = np.arange(miny, maxy, grado_cuad)

celdas = [] #Lista donde guardaremos las coordenadas de las celdas

for x in x_coords:
    for y in y_coords:
        poligono = Polygon([(x, y), 
        (x + grado_cuad, y), 
        (x + grado_cuad, y + grado_cuad), 
        (x, y + grado_cuad)])
        celdas.append(poligono)
        
celdas[:5] #Se muestra solo los primeros 5 resultados

[<POLYGON ((-117.128 7.221, -116.128 7.221, -116.128 8.221, -117.128 8.221, -...>,
 <POLYGON ((-117.128 8.221, -116.128 8.221, -116.128 9.221, -117.128 9.221, -...>,
 <POLYGON ((-117.128 9.221, -116.128 9.221, -116.128 10.221, -117.128 10.221,...>,
 <POLYGON ((-117.128 10.221, -116.128 10.221, -116.128 11.221, -117.128 11.22...>,
 <POLYGON ((-117.128 11.221, -116.128 11.221, -116.128 12.221, -117.128 12.22...>]

Seguidamente se prosigue a crear el GeoDataFrame en donde se van a guardar las variables de interes, debido a que algunos cálculos posteriores toman mucho tiempo dentro de esta base solo se van a agregar las variables que se utilizan para calcular el CHF. Posteriormente se creara una base en la cual si se hace los cálculos del CHF.

In [121]:
grid_gdf = gpd.GeoDataFrame({'geometry': celdas}, 
                            crs=paises.crs)

In [122]:
# Calcular la frecuencia de huracanes en cada cuadrícula

grid_gdf['frecuencia_h'] = grid_gdf['geometry'].apply(lambda geom: len(huracanes[huracanes.within(geom)]))

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [123]:
# Calcular el promedio de velocidad de huracanes en cada cuadrícula

grid_gdf['velocidad_promedio'] = grid_gdf['geometry'].apply(lambda geom: huracanes[huracanes.within(geom)]['MaxW'].mean())

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [124]:
# Calcular la frecuencia de huracanes en cada cuadrícula que no pasa por ahí, N_i

grid_gdf['frecuencia_ni'] = grid_gdf['geometry'].apply(lambda geom: len(huracanes[~huracanes.within(geom)]))

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [125]:
# Calcular la precipitacion en la cuadricula entre la promedio

grid_gdf['Precipitacion'] = grid_gdf['geometry'].apply(lambda geom: 
                                                       precipitacion[precipitacion.within(geom)]['Precipitation'].mean()/ precipitacion['Precipitation'].mean())

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [126]:
# Calcular el Viento en la cuadricula entre el promedio del viento

grid_gdf['Wind'] = grid_gdf['geometry'].apply(lambda geom: 
                                                       wind[wind.within(geom)]['Wind'].mean()/ wind['Wind'].mean())

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [127]:
# Calcular la Velocidad promedio pero sin contar esa cuadrícula, por decirlo de otra manera es como calcular la velocidad que tiene el
# CHF pero la inversa puesto que estamos calculando la frecuencia de los huracanes que no pasan por la cuadricula entonces seria logico
# Calcular la velocidad promedio de los huracanes que no pasan por la cuadricula. De otra manera el indice da muy alto, posteriormente
# Se aprecia el indice sin el uso de esta variable

grid_gdf['Velocidad_promedio_NC'] = grid_gdf['geometry'].apply(lambda geom: huracanes[~huracanes.within(geom)]['MaxW'].mean())

/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/geopandas/geoseries.py:574: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


Posteriormente de la creación de la base de datos solo con coordenadas y las variables de interes para el cálculo de los CHF, hacemos un preview de la data para observar cuales variables tenemos y posterior a eso se guarda la base en formato csv y geopandas.

In [131]:
grid_gdf.tail() #Preview de la base

,geometry,frecuencia_h,velocidad_promedio,Precipitacion,frecuencia_ni,Wind,Velocidad_promedio_NC
1035,"POLYGON ((-78.12776 28.22054, -77.12776 28.220...",2,47.900000,NaN,409,1.136180,45.129584
1036,"POLYGON ((-78.12776 29.22054, -77.12776 29.220...",2,26.300000,NaN,409,NaN,45.235208
1037,"POLYGON ((-78.12776 30.22054, -77.12776 30.220...",3,31.666667,NaN,408,0.546354,45.242157
1038,"POLYGON ((-78.12776 31.22054, -77.12776 31.220...",6,36.450000,NaN,405,0.630720,45.271852
1039,"POLYGON ((-78.12776 32.22054, -77.12776 32.220...",0,NaN,NaN,411,NaN,45.143066


In [132]:
#Guardar en formato csv

grid_gdf.to_csv("Base_variables_CHF.csv") 

#Guardar en formato geojson

with open('Base_variables_CHF.geojson' , 'w') as file:
    file.write(grid_gdf.to_json())

### Segunda Parte

Dentro de esta sub-sección de procede a crear ya los indices CHF, tanto los nuevos como los viejos. Además se observara la incosistencia que tiene el CHF nuevo propuesto por los investigadores.

In [133]:
CHF = grid_gdf.copy() #Creamos una copia de la base anterior para hacer los cálculos

CHF['Indicador_CHF'] = CHF['frecuencia_h']/(CHF['velocidad_promedio']*24) #Cálculo del CHF viejo

CHF['Indicador_CHF'] = CHF['Indicador_CHF'].fillna(0) #Inputamos 0 en el indicador CHF los cuales tienen NAN para futuros calculos.

In [135]:
CHF.tail() #Preview de la base CHF

,geometry,frecuencia_h,velocidad_promedio,Precipitacion,frecuencia_ni,Wind,Velocidad_promedio_NC,Indicador_CHF
1035,"POLYGON ((-78.12776 28.22054, -77.12776 28.220...",2,47.900000,NaN,409,1.136180,45.129584,0.001740
1036,"POLYGON ((-78.12776 29.22054, -77.12776 29.220...",2,26.300000,NaN,409,NaN,45.235208,0.003169
1037,"POLYGON ((-78.12776 30.22054, -77.12776 30.220...",3,31.666667,NaN,408,0.546354,45.242157,0.003947
1038,"POLYGON ((-78.12776 31.22054, -77.12776 31.220...",6,36.450000,NaN,405,0.630720,45.271852,0.006859
1039,"POLYGON ((-78.12776 32.22054, -77.12776 32.220...",0,NaN,NaN,411,NaN,45.143066,0.000000


In [136]:
#Creación del Indicador modificado, con el efecto indirecto (hecho por los investigadores)

CHF['Indicador_CHF_Mod'] = CHF['Indicador_CHF'] + (CHF['frecuencia_ni']/ (CHF['velocidad_promedio']*24))*CHF['Precipitacion']*CHF['Wind']

In [137]:
CHF.tail()

,geometry,frecuencia_h,velocidad_promedio,Precipitacion,frecuencia_ni,Wind,Velocidad_promedio_NC,Indicador_CHF,Indicador_CHF_Mod
1035,"POLYGON ((-78.12776 28.22054, -77.12776 28.220...",2,47.900000,NaN,409,1.136180,45.129584,0.001740,NaN
1036,"POLYGON ((-78.12776 29.22054, -77.12776 29.220...",2,26.300000,NaN,409,NaN,45.235208,0.003169,NaN
1037,"POLYGON ((-78.12776 30.22054, -77.12776 30.220...",3,31.666667,NaN,408,0.546354,45.242157,0.003947,NaN
1038,"POLYGON ((-78.12776 31.22054, -77.12776 31.220...",6,36.450000,NaN,405,0.630720,45.271852,0.006859,NaN
1039,"POLYGON ((-78.12776 32.22054, -77.12776 32.220...",0,NaN,NaN,411,NaN,45.143066,0.000000,NaN


**Nota:** Hay un problema en el cálculo del CHF modificado en la parte del efecto indirecto. Esto se debe a que el efecto indirecto se mide en la cuadricula con la velocidad de traslación del huracan que pasa por dicha cuadrícula pero creo que debería ser esa v, la velocidad de traslación promedio de los demás huracánes sin esa cuadrícula. CONSULTAR EN LA PROXIMA REUNION

Si se hace lo planteado de mi parte el CHF_modificado quedaría de la siguiente manera. 

In [138]:
#Creación del Indicador modificado propuesto por mi persona, con el efecto indirecto 

CHF['Indicador_CHF_Mod2'] = CHF['Indicador_CHF'] + (CHF['frecuencia_ni']/ (CHF['Velocidad_promedio_NC']*24))*CHF['Precipitacion']*CHF['Wind']

In [141]:
CHF = CHF[['geometry','Indicador_CHF','Indicador_CHF_Mod','Indicador_CHF_Mod2']]

Por último se guarda la base de datos del CHF en formato csv y formato geojson para posteriores analisis si los requiere.

In [142]:
#Guardar en formato csv

CHF.to_csv("CHF_mod.csv") 

#Guardar en formato geojson

with open('CHF_mod.geojson' , 'w') as file:
    file.write(CHF.to_json())